In [ ]:
from src import DS_Generator
import tensorflow as tf
from util import TRAIN_SPECIES_DF, TEST_DATA_PATH
import tqdm
import numpy as nptest
from scipy.spatial.distance import cdist
import pandas as pd
from src.models import return_soft_max_pretrained_siamese_model

In [1]:
solutions_folder = "path_for_our_solutions"
distance_for_new_whales = 0.35

In [ ]:
test_df = pd.DataFrame(os.listdir(TEST_DATA_PATH),columns=["image"])
test_df["label"]=0
test_ds = DS_Generator.DS_Generator().return_plain_ds(test_df,64,train_imgs = False)
whole_train_ds = DS_Generator.DS_Generator().return_plain_ds(TRAIN_SPECIES_DF,64)

In [ ]:
# some model you want to try
model = return_soft_max_pretrained_siamese_model()

In [ ]:
train_embedding = model.predict(whole_train_ds,verbose=1)
test_embedding = model.predict(test_ds,verbose=1)

# save just in case
np.save(solutions_folder + "/train_embedding.npy",train_embedding)
np.save(solutions_folder + "/test_embedding.npy",test_embedding )


# compute the pairwise_distance (this will take long)
pairwise_distance = cdist(test_embedding,train_embedding) # distance_‘cosine’
# and also save it just in case
np.save(solutions_folder + "/pairwise_distance.npy",pairwise_distance )



# split it up to not overstress memory
split_ups = np.ceil(np.linspace(0,27956)).astype(int)
closest_vals, closest_indices = list(),list()
for i in tqdm.tqdm(range(1,len(split_ups))):
    split = pairwise_distance[split_ups[i-1]:split_ups[i]]
    vals , indeces = tf.nn.top_k(tf.math.negative(split),5)
    vals = tf.math.negative(vals)
    closest_vals.append(vals)
    closest_indices.append(indeces)


vals = np.concatenate(list1)
indeces = np.concatenate(list2)

for i in tqdm.tqdm(range(len(test_df))):
    distances = vals[i]
    labels = [TRAIN_SPECIES_DF.iloc[k,2] for k in indeces[i]]
    
    args = np.argwhere(distances > distance_for_new_whales)
    if len(args)!=0:
        first = int(args[0,0])
        labels.insert(first,"new_individual")

    test_df.iloc[i,-1]= " ".join(labels[:5])


test_df.to_csv(solutions_folder + "/submission.csv",index=False,columns=["image","predictions"])